In [ ]:
from PIL import Image
from keras.applications import ResNet50
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import os

# Paths to crack and non-crack image folders
crack_path = 'C:\\New folder\\Dr. Surya\\Project\\crack'
noncrack_path = 'C:\\New folder\\Dr. Surya\\Project\\noncrack'

# Define target size
TARGET_SIZE = (224, 224) 

# Load all image files and annotations  
crack_images = [(os.path.join(crack_path, f), 1) for f in os.listdir(crack_path)] 
noncrack_images = [(os.path.join(noncrack_path, f), 0) for f in os.listdir(noncrack_path)]

images = crack_images + noncrack_images
np.random.shuffle(images)

# Preprocess and extract features  
def preprocess(image):  
    img = Image.open(image)
    img = img.resize(TARGET_SIZE)  
    x = np.array(img)/255

    return x, label

X = []
y = []

for img, label in images:
    x, label = preprocess(img)
  
    X.append(x)
    y.append(label)

# Split train/test  
X_train, X_test, y_train, y_test = train_test_split(X, y)

# One-hot encode labels
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

# Resize images to a consistent size
resized_images_train = []
for img in X_train:
    resized_img = cv2.resize(img, (224, 224))
    resized_img = cv2.convertScaleAbs(resized_img) # Convert to CV_8U format
    resized_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB) # Convert to RGB format
    resized_images_train.append(resized_img)

resized_images_test= []
for img in X_test:
    resized_img = cv2.resize(img, (224, 224))
    resized_img= cv2.convertScaleAbs(resized_img) # Convert to CV_8U format
    resized_img= cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB) # Convert to RGB format
    resized_images_test.append(resized_img)

# Convert resized_images to NumPy array
X_train = np.empty((len(resized_images_train), 224, 224, 3), dtype=np.float32)
for i, img in enumerate(resized_images_train):
    X_train[i] = img

X_test = np.empty((len(resized_images_test), 224, 224, 3), dtype=np.float32)
for i, img in enumerate(resized_images_test):
    X_test[i] = img

y_train = np.array(y_train)
y_test = np.array(y_test)

# One-hot encode labels
y_train_cat = to_categorical(y_train, num_classes=2)
y_test_cat = to_categorical(y_test, num_classes=2)

# Create model
base_model = ResNet50(weights='imagenet', include_top=False, 
                input_shape=(224,224,3))
x = base_model.output  
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
preds = Dense(2, activation='softmax')(x)

model = Model(base_model.input, preds)

# Train model
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=1e-4), metrics=['accuracy'])
checkpoint = ModelCheckpoint('C:\\New folder\\Dr. Surya\\Project\\CNN2.h5', save_best_only=True)
print(y_train_cat.shape)
print(model.predict(X_train).shape)
model.fit(X_train, y_train_cat, epochs=20, validation_data=(X_test,y_test_cat), 
          callbacks=[checkpoint])

# Load best model and evaluate
model.load_weights('C:\\New folder\\Dr. Surya\\Project\\CNN2.h5')
loss, accuracy = model.evaluate(X_test, y_test)
print('Test accuracy:', accuracy)